# Logistic Regression with Numpy Implementation

## 1. Introduction

Logistic regression is one of the most fundamental machine learning models for binary classification. I will summarize its methodology and implement it from scratch using NumPy.

The problem we solve is **binary classification,** for example, the doctor would like to base on patients's features, including mean radius, mean texture, etc, to classify breat cancer into one of the following two case:

- "malignant":  𝑦=1 
- "benign":  𝑦=0 

which correspond to serious and gentle case respectively.

We will load the breast cancer data from scikit-learn as a toy dataset, and split the data into the training and test datasets.

## 2. Logistic Regression Model

[To be continued.]

## 3. Numpy Implementation of Logistic Regressio

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random
import numpy as np

import tensorflow as tf

np.random.seed(71)

In [ ]:
class LogisticRegression(object):
    """Numpy implementation of Logistic Regression."""
    def __init__(self, batch_size=64, lr=0.01, n_epochs=1000):
        self.batch_size = batch_size
        self.lr = lr
        self.n_epochs = n_epochs

    def get_dataset(self, X_train, y_train, shuffle=True):
        """Get dataset and information."""
        self.X_train = X_train
        self.y_train = y_train

        # Get the numbers of examples and inputs.
        self.n_examples, self.n_inputs = self.X_train.shape

        if shuffle:
            idx = list(range(self.n_examples))
            random.shuffle(idx)
            self.X_train = self.X_train[idx]
            self.y_train = self.y_train[idx]

    def _create_weights(self):
        """Create model weights and bias."""
        self.w = np.zeros(self.n_inputs).reshape(self.n_inputs, 1)
        self.b = np.zeros(1).reshape(1, 1)

    def _logit(self, X):
        """Logit: unnormalized log probability."""
        return np.matmul(X, self.w) + self.b

    def _sigmoid(self, logit):
        """Sigmoid function by stabilization trick.

        sigmoid(z) = 1 / (1 + exp(-z)) 
                   = exp(z) / (1 + exp(z)) * exp(z_max) / exp(z_max)
                   = exp(z - z_max) / (exp(-z_max) + exp(z - z_max)),
        where z is the logit, and z_max = z - max(0, z).
        """
        logit_max = np.maximum(0, logit)
        logit_stable = logit - logit_max
        return np.exp(logit_stable) / (np.exp(-logit_max) + np.exp(logit_stable))
    
    def _model(self, X):
        """Logistic regression model."""
        logit = self._logit(X)
        return self._sigmoid(logit)

    def _loss(self, y, logit):
        """Cross entropy loss by stabilizaiton trick.

        cross_entropy_loss(y, z) 
          = - 1/n * \sum_{i=1}^n y_i * log p(y_i = 1|x_i) + (1 - y_i) * log p(y_i = 0|x_i)
          = - 1/n * \sum_{i=1}^n y_i * (z_i - log(1 + exp(z_i))) + (1 - y_i) * (-log(1 + exp(z_i))),
        where z is the logit, z_max = z - max(0, z),
          log p(y = 1|x)
            = log (1 / (1 + exp(-z))) 
            = log (exp(z) / (1 + exp(z)))
            = z - log(1 + exp(z))
        and 
          log(1 + exp(z)) := logsumexp(z)
            = log(exp(0) + exp(z))
            = log(exp(0) + exp(z) * exp(z_max) / exp(z_max))
            = z_max + log(exp(-z_max) + exp(z - z_max)).
        """
        logit_max = np.maximum(0, logit)
        logit_stable = logit - logit_max
        logsumexp_stable = logit_max + np.log(np.exp(-logit_max) + np.exp(logit_stable))
        self.cross_entropy = -(y * (logit - logsumexp_stable) + (1 - y) * (-logsumexp_stable))
        return np.mean(self.cross_entropy)

    def _optimize(self, X, y):
        """Optimize by stochastic gradient descent."""
        m = X.shape[0]

        y_hat = self._model(X) 
        dw = 1 / m * np.matmul(X.T, y_hat - y)
        db = np.mean(y_hat - y)

        for (param, grad) in zip([self.w, self.b], [dw, db]):
            param[:] = param - self.lr * grad

    def _fetch_batch(self):
        """Fetch batch dataset."""
        idx = list(range(self.n_examples))
        for i in range(0, self.n_examples, self.batch_size):
            idx_batch = idx[i:min(i + self.batch_size, self.n_examples)]
            yield (self.X_train.take(idx_batch, axis=0), self.y_train.take(idx_batch, axis=0))

    def fit(self):
        """Fit model."""
        self._create_weights()

        for epoch in range(self.n_epochs):
            total_loss = 0
            for X_train_b, y_train_b in self._fetch_batch():
                y_train_b = y_train_b.reshape((y_train_b.shape[0], -1))
                self._optimize(X_train_b, y_train_b)
                train_loss = self._loss(y_train_b, self._logit(X_train_b))
                total_loss += train_loss * X_train_b.shape[0]

            if epoch % 100 == 0:
                print('epoch {0}: training loss {1}'.format(epoch, total_loss / self.n_examples))

        return self

    def get_coeff(self):
        return self.b, self.w.reshape((-1,))

    def predict(self, X):
        return self._model(X).reshape((-1,))

## 4. Data Preparation and Preprocessing

In [2]:
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression as LogisticRegressionSklearn

# https://github.com/bowen0701/machine-learning/blob/master/metrics.py
from metrics import accuracy

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Read breast cancer data.
X, y = load_breast_cancer(return_X_y=True)

In [5]:
X.shape, y.shape

((569, 30), (569,))

In [6]:
X[:3]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e

In [7]:
y[:3]

array([0, 0, 0])

In [117]:
# Split data into training and test datasets.
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=71, shuffle=True, stratify=y)

In [118]:
print(X_train_raw.shape, y_train.shape)
print(X_test_raw.shape, y_test.shape)

(426, 30) (426,)
(143, 30) (143,)


In [119]:
# Feature engineering for standardizing features by min-max scaler.
min_max_scaler = MinMaxScaler()

X_train = min_max_scaler.fit_transform(X_train_raw)
X_test = min_max_scaler.transform(X_test_raw)

## 4. Fitting Logistic Regression Model in NumPy

In [ ]:
# Fit our Logistic Regression.
logreg = LogisticRegression(batch_size=64, lr=1, n_epochs=1000)

In [ ]:
# Get datasets and build graph.
logreg.get_dataset(X_train, y_train, shuffle=True)

In [ ]:
logreg.fit()

In [ ]:
# Get coefficient.
logreg.get_coeff()

In [ ]:
# Predicted probabilities for training data.
p_train_hat = logreg.predict(X_train)
p_train_hat[:3]

In [ ]:
# Predicted labels for training data.
y_train_hat = (p_train_hat > 0.5) * 1
y_train_hat[:3]

In [ ]:
# Prediction accuracy for training data.
accuracy(y_train, y_train_hat)

In [ ]:
# Predicted label correctness for test data.
p_test_hat = logreg.predict(X_test)
y_test_hat = (p_test_hat > 0.5) * 1

In [ ]:
# Prediction accuracy for test data.
accuracy(y_test, y_test_hat)

## Fitting Logistic Regression Model in TensorFlow

In [128]:
def reset_tf_graph(seed=71):
    """Reset default TensorFlow graph."""
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


class LogisticRegressionTF(object):
    """A TensorFlow implementation of Logistic Regression."""
    def __init__(self, batch_size=64, learning_rate=0.01, n_epochs=1000):
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate

    def get_dataset(self, X_train, y_train, shuffle=True):
        """Get dataset and information."""
        self.X_train = X_train
        self.y_train = y_train

        # Get the numbers of examples and inputs.
        self.n_examples, self.n_inputs = self.X_train.shape

        idx = list(range(self.n_examples))
        if shuffle:
            random.shuffle(idx)
        self.X_train = self.X_train[idx]
        self.y_train = self.y_train[idx]

    def _create_placeholders(self):
        """Create placeholder for features and labels."""
        self.X = tf.placeholder(tf.float32, shape=(None, self.n_inputs), name='X')
        self.y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

    def _create_weights(self):
        """Create and initialize model weights and bias."""
        self.w = tf.get_variable(name='weights',
                                 shape=(self.n_inputs, 1),
                                 initializer=tf.random_normal_initializer())
        self.b = tf.get_variable(name='bias', 
                                 shape=(1, 1),
                                 initializer=tf.zeros_initializer())

    def _create_model(self):
        # Create logistic regression model.
        self.logits = tf.matmul(self.X, self.w) + self.b

    def _create_loss(self):
        # Create cross entropy loss.
        self.cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=self.y,
            logits=self.logits,
            name='cross_entropy')
        self.loss = tf.reduce_mean(self.cross_entropy, name='loss')

    def _create_optimizer(self):
        # Create gradient descent optimization.
        # self.optimizer = (
        #     tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate)
        #     .minimize(self.loss))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)


    def build_graph(self):
        """Build computational graph."""
        self._create_placeholders()
        self._create_weights()
        self._create_model()
        self._create_loss()
        self._create_optimizer()

    def _fetch_batch(self):	
        """Fetch batch dataset."""
        idx = list(range(self.n_examples))
        for i in range(0, self.n_examples, self.batch_size):
            idx_batch = idx[i:min(i + self.batch_size, self.n_examples)]
            yield (self.X_train[idx_batch, :], self.y_train[idx_batch].reshape(-1, 1))

    def fit(self):
        """Fit model."""
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            for epoch in range(self.n_epochs):
                total_loss = 0
                for X_train_b, y_train_b in self._fetch_batch():
                    feed_dict = {self.X: X_train_b, self.y: y_train_b}
                    _, batch_loss = sess.run([self.optimizer, self.loss],
                                             feed_dict=feed_dict)
                    total_loss += batch_loss * X_train_b.shape[0]

                if epoch % 100 == 0:
                    print('Epoch {0}: training loss: {1}'
                          .format(epoch, total_loss / self.n_examples))

    def get_coeff(self):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return self.b.eval(), self.w.eval().reshape((-1,))

    def predict(self, X):
        # return self._model(X).reshape((-1,))
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            logit = tf.matmul(X, self.w) + self.b
            return tf.math.sigmoid(logit).eval()

In [136]:
reset_tf_graph()
logreg_tf = LogisticRegressionTF(batch_size=64, learning_rate=0.5, n_epochs=1000)

In [137]:
logreg_tf.get_dataset(X_train, y_train, shuffle=True)

In [138]:
logreg_tf.build_graph()

In [139]:
logreg_tf.fit()

Epoch 0: training loss: 0.8151776698273672
Epoch 100: training loss: 0.047429754525562966
Epoch 200: training loss: 0.040094848389916574
Epoch 300: training loss: 0.03558398062634356
Epoch 400: training loss: 0.031960097014764106
Epoch 500: training loss: 0.028774834329813297
Epoch 600: training loss: 0.025468632264036527
Epoch 700: training loss: 0.025660875485116887
Epoch 800: training loss: 0.021246203645504138
Epoch 900: training loss: 0.0220542639458963


In [140]:
logreg_tf.get_coeff()

(array([[0.]], dtype=float32),
 array([ 1.0636256 , -0.6662474 ,  0.5206876 ,  0.6081255 ,  0.38066953,
        -0.76240456,  0.08705895, -1.4782716 ,  0.5149946 ,  0.05717897,
         1.7587228 ,  1.1775416 , -0.41634285, -0.27666384, -0.8869958 ,
        -0.44037002,  0.8534515 ,  0.54841787,  0.02681548,  0.7779246 ,
         0.30678782,  0.96707946,  0.01468348,  0.19701141,  0.58284706,
        -0.33996764, -0.08145858,  0.5403245 , -0.20620511,  1.1788939 ],
       dtype=float32))

In [141]:
# Predicted probabilities for training data.
p_train_hat = logreg_tf.predict((tf.cast(X_train, dtype=tf.float32)))
print(p_train_hat[:10])

# Predicted labels for training data.
y_train_hat = (p_train_hat > 0.5) * 1
print(y_train_hat[:10])

# Prediction accuracy for training data.
accuracy(y_train, y_train_hat)

[[0.7878543 ]
 [0.9469813 ]
 [0.892252  ]
 [0.8097632 ]
 [0.81836   ]
 [0.81464463]
 [0.86282134]
 [0.89409757]
 [0.7902181 ]
 [0.8816503 ]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


0.6267605633802817

In [142]:
# Predicted probabilities for test data.
p_test_hat = logreg_tf.predict((tf.cast(X_test, dtype=tf.float32)))
print(p_test_hat[:3])

# Predicted labels for training data.
y_test_hat = (p_test_hat > 0.5) * 1
y_test_hat[:3]

# Prediction accuracy for training data.
accuracy(y_test, y_test_hat)

[[0.92556256]
 [0.76511514]
 [0.88554823]]


0.6293706293706294

## 5. Fitting Sklearn's Logistic Regression as Benchmark

In [38]:
# Fit sklearn's Logistic Regression.
logreg_sk = LogisticRegressionSklearn(C=1e4, solver='lbfgs', max_iter=500)

logreg_sk.fit(X_train, y_train)

LogisticRegression(C=10000.0, max_iter=500)

In [39]:
# Get coefficients.
logreg_sk.intercept_, logreg_sk.coef_

(array([56.06250509]),
 array([[  53.5460616 ,  -27.2575739 ,   48.30697654,   10.5636878 ,
          -14.75837806,   98.5009966 ,  -52.51936527,  -52.16906591,
           -5.08742246,  -53.96348797,  -33.97198842,   -5.48905184,
          -19.38885928,  -43.89981909,   38.75665922,  -51.43678914,
           83.21007672,  -21.89925037,   14.96797392,   79.99757062,
          -59.04206865,   -3.91791317,  -63.58395555, -103.96747709,
           -7.9699581 ,   20.04904076,  -21.96650031,  -21.30939901,
          -21.55187209,  -11.69936363]]))

In [40]:
# Predicted labels for training data.
p_train_hat = logreg_sk.predict(X_train)
p_train_hat[:3]

array([1, 0, 0])

In [41]:
y_train_hat = (p_train_hat > 0.5) * 1

In [42]:
# Predicted label correctness for training data.
# y_pred_train == y_train

In [43]:
# Prediction accuracy for training data.
accuracy(y_train, y_train_hat)

1.0

In [44]:
# Predicted label correctness for test data.
p_test_hat = logreg_sk.predict(X_test)
y_test_hat = (p_test_hat > 0.5) * 1

In [45]:
# # Prediction accuracy for test data.
accuracy(y_test, y_test_hat)

0.965034965034965